# Anticipez les besoins en consommation électrique de bâtiments
Nous travaillons pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, notre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation individuelle. Des relevés minutieux ont été effectués en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, nous voulons tenter de prédire les émissions de CO2 et la consommation totale d’énergie des bâtiments pour lesquels elles n’ont pas encore été mesurées. </br>
Dans ce notebook, nous testerons différents modèles de prédiction des émissions de CO2 des bâtiments de la ville de Seattle.

# Sommaire :
- **Partie 1:** <a href="#C1">Importation des données</a>
- **Partie 2:** <a href="#C2">Méthodes de Régression</a>
    - <a href="#C3">Régressions naïves</a>
    - <a href="#C4">Régression Linéaire</a>
    - <a href="#C5">Régression Ridge</a>
    - <a href="#C6">Régression Lasso</a>
    - <a href="#C7">Régression ElasticNet</a>
- **Partie 3:** <a href="#C8">Méthodes ensemblistes</a>
    - <a href="#C9">Bagging</a>
    - <a href="#C10">Random Forest</a>
    - <a href="#C11">Gradient Boosting</a>
- **Partie 4:** <a href="#C12">Méthodes non linéaires</a>
    - <a href="#C13"> SVM à noyau</a>
    - <a href="#C14">Régression ridge à noyau</a>
- **Partie 5:** <a href="#C15">Sélection du modèle de prédiction</a>

# <a name="C1">Partie 1: Importation des données</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import datasets
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, LassoCV, Lasso, ElasticNetCV, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.dummy import DummyRegressor
from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

pd.set_option('display.max_columns', 100)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
#Jeu de données nettoyé sans transformation
data_original = pd.read_csv('dataNettoye.csv',sep=',',encoding='utf-8')
data = data_original.copy()
data.head()

,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,ENERGYSTARScore,SiteEnergyUse(kBtu),GHGEmissions(MetricTonsCO2e),BuildingAge,BuildingType_Multifamily HR (10+),BuildingType_Multifamily LR (1-4),BuildingType_Multifamily MR (5-9),BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Distribution Center\n,PrimaryPropertyType_High-Rise Multifamily,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mid-Rise Multifamily,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Non-Refrigerated Warehouse,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Residence Hall/Dormitory,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Restaurant\n,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Self-Storage Facility\n,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Supermarket/Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,...,SecondLargestPropertyUseType_Repair Services (Vehicle,SecondLargestPropertyUseType_Residence Hall/Dormitory,SecondLargestPropertyUseType_Restaurant,SecondLargestPropertyUseType_Retail Store,SecondLargestPropertyUseType_Self-Storage Facility,SecondLargestPropertyUseType_Senior Care Community,SecondLargestPropertyUseType_Shoe,SecondLargestPropertyUseType_Social/Meeting Hall,SecondLargestPropertyUseType_Strip Mall,SecondLargestPropertyUseType_Supermarket/Grocery Store,SecondLargestPropertyUseType_Swimming Pool,SecondLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_Financial Office,ThirdLargestPropertyUseType_Locksmith,ThirdLargestPropertyUseType_Manufacturing/Industrial Plant,ThirdLargestPropertyUseType_Medical Office,ThirdLargestPropertyUseType_Movie Theater,ThirdLargestPropertyUseType_Multifamily Housing,ThirdLargestPropertyUseType_Museum,ThirdLargestPropertyUseType_NoThirdUse,ThirdLargestPropertyUseType_Non-Refrigerated Warehouse,ThirdLargestPropertyUseType_Office,ThirdLargestPropertyUseType_Other,ThirdLargestPropertyUseType_Other - Education,ThirdLargestPropertyUseType_Other - Entertainment/Public Assembly,ThirdLargestPropertyUseType_Other - Lodging/Residential,ThirdLargestPropertyUseType_Other - Mall,ThirdLargestPropertyUseType_Other - Public Services,ThirdLargestPropertyUseType_Other - Recreation,ThirdLargestPropertyUseType_Other - Restaurant/Bar,ThirdLargestPropertyUseType_Other - Services,ThirdLargestPropertyUseType_Other - Technology/Science,ThirdLargestPropertyUseType_Outpatient Rehabilitation/Physical Therapy,ThirdLargestPropertyUseType_Parking,ThirdLargestPropertyUseType_Personal Services (Health/Beauty,ThirdLargestPropertyUseType_Pre-school/Daycare,ThirdLargestPropertyUseType_Refrigerated Warehouse,ThirdLargestPropertyUseType_Repair Services (Vehicle,ThirdLargestPropertyUseType_Residence Hall/Dormitory,ThirdLargestPropertyUseType_Restaurant,ThirdLargestPropertyUseType_Retail Store,ThirdLargestPropertyUseType_Self-Storage Facility,ThirdLargestPropertyUseType_Senior Care Community,ThirdLargestPropertyUseType_Shoe,ThirdLargestPropertyUseType_Social/Meeting Hall,ThirdLargestPropertyUseType_Supermarket/Grocery Store,ThirdLargestPropertyUseType_Swimming Pool,ThirdLargestPropertyUseType_Vocational School,ThirdLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_etc)
0,1927,1,1,88434,65.0,6981428.0,249.43,88,0,0,0,1,0,0,0,0,0

In [3]:
#Jeu de données nettoyé avec transformations
dataTransfo_original = pd.read_csv('dataNettoyeTransforme.csv',sep=',',encoding='utf-8')
dataLog = dataTransfo_original.copy()
dataLog.head()

,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,ENERGYSTARScore,SiteEnergyUse(kBtu),GHGEmissions(MetricTonsCO2e),BuildingAge,BuildingType_Multifamily HR (10+),BuildingType_Multifamily LR (1-4),BuildingType_Multifamily MR (5-9),BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Distribution Center\n,PrimaryPropertyType_High-Rise Multifamily,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mid-Rise Multifamily,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Non-Refrigerated Warehouse,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Residence Hall/Dormitory,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Restaurant\n,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Self-Storage Facility\n,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Supermarket/Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,...,SecondLargestPropertyUseType_Repair Services (Vehicle,SecondLargestPropertyUseType_Residence Hall/Dormitory,SecondLargestPropertyUseType_Restaurant,SecondLargestPropertyUseType_Retail Store,SecondLargestPropertyUseType_Self-Storage Facility,SecondLargestPropertyUseType_Senior Care Community,SecondLargestPropertyUseType_Shoe,SecondLargestPropertyUseType_Social/Meeting Hall,SecondLargestPropertyUseType_Strip Mall,SecondLargestPropertyUseType_Supermarket/Grocery Store,SecondLargestPropertyUseType_Swimming Pool,SecondLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_Financial Office,ThirdLargestPropertyUseType_Locksmith,ThirdLargestPropertyUseType_Manufacturing/Industrial Plant,ThirdLargestPropertyUseType_Medical Office,ThirdLargestPropertyUseType_Movie Theater,ThirdLargestPropertyUseType_Multifamily Housing,ThirdLargestPropertyUseType_Museum,ThirdLargestPropertyUseType_NoThirdUse,ThirdLargestPropertyUseType_Non-Refrigerated Warehouse,ThirdLargestPropertyUseType_Office,ThirdLargestPropertyUseType_Other,ThirdLargestPropertyUseType_Other - Education,ThirdLargestPropertyUseType_Other - Entertainment/Public Assembly,ThirdLargestPropertyUseType_Other - Lodging/Residential,ThirdLargestPropertyUseType_Other - Mall,ThirdLargestPropertyUseType_Other - Public Services,ThirdLargestPropertyUseType_Other - Recreation,ThirdLargestPropertyUseType_Other - Restaurant/Bar,ThirdLargestPropertyUseType_Other - Services,ThirdLargestPropertyUseType_Other - Technology/Science,ThirdLargestPropertyUseType_Outpatient Rehabilitation/Physical Therapy,ThirdLargestPropertyUseType_Parking,ThirdLargestPropertyUseType_Personal Services (Health/Beauty,ThirdLargestPropertyUseType_Pre-school/Daycare,ThirdLargestPropertyUseType_Refrigerated Warehouse,ThirdLargestPropertyUseType_Repair Services (Vehicle,ThirdLargestPropertyUseType_Residence Hall/Dormitory,ThirdLargestPropertyUseType_Restaurant,ThirdLargestPropertyUseType_Retail Store,ThirdLargestPropertyUseType_Self-Storage Facility,ThirdLargestPropertyUseType_Senior Care Community,ThirdLargestPropertyUseType_Shoe,ThirdLargestPropertyUseType_Social/Meeting Hall,ThirdLargestPropertyUseType_Supermarket/Grocery Store,ThirdLargestPropertyUseType_Swimming Pool,ThirdLargestPropertyUseType_Vocational School,ThirdLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_etc)
0,7.564238,1,1,11.390023,4.189655,15.758764,5.523179,4.4886

Une des problèmatiques est d'évaluer l'intérêt de la varibale `ENERGYSTARScore` dans la prédiction des émissions de gaz à effet de serre. On va donc créer une copie du jeu de données sans cette variable afin de pouvoir comparer les différents modèles de prédiction avec ou sans la variable `ENERGYSTARScore`.

In [4]:
dataSE = data.drop(['ENERGYSTARScore'], axis=1)
dataLogSE = dataLog.drop(['ENERGYSTARScore'], axis=1)

On peut à présent préparer notre jeu d'entraînement qui servira pour entrainer les différents modèles que l'on va traiter, et notre jeu de test qui servira à comparer les performances de généralisation des différents modèles.

In [5]:
#Pour le jeu non transformé

#Jeux d'entraînement et de test sans la varibale ENERGYSTARScore
X_SE = dataSE.drop(['GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)'], axis=1)
y_SE = dataSE['GHGEmissions(MetricTonsCO2e)']
X_SE_train, X_SE_test, y_SE_train, y_SE_test = train_test_split(X_SE, y_SE, test_size = 0.2)

#Jeux d'entraînement et de test avec la varibale ENERGYSTARScore
X = data.drop(['GHGEmissions(MetricTonsCO2e)'], axis=1)
y = data['GHGEmissions(MetricTonsCO2e)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#Standardisation des données d'entraînement
std_scale = StandardScaler().fit(X_SE_train)
X_SE_train_std = std_scale.transform(X_SE_train)
X_SE_test_std = std_scale.transform(X_SE_test)

std_scale = StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

In [6]:
#Pour le jeu transformé

#Jeux d'entraînement et de test sans la varibale ENERGYSTARScore
X_log_SE = dataLogSE.drop(['GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)'], axis=1)
y_log_SE = dataLogSE['GHGEmissions(MetricTonsCO2e)']
X_log_SE_train, X_log_SE_test, y_log_SE_train, y_log_SE_test = train_test_split(X_log_SE, y_log_SE, test_size = 0.2)

#Jeux d'entraînement et de test avec la varibale ENERGYSTARScore
X_log = dataLog.drop(['GHGEmissions(MetricTonsCO2e)'], axis=1)
y_log = dataLog['GHGEmissions(MetricTonsCO2e)']
X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y_log, test_size = 0.2)

#Standardisation des données d'entraînement
std_scale = StandardScaler().fit(X_log_SE_train)
X_log_SE_train_std = std_scale.transform(X_log_SE_train)
X_log_SE_test_std = std_scale.transform(X_log_SE_test)

std_scale = StandardScaler().fit(X_log_train)
X_log_train_std = std_scale.transform(X_log_train)
X_log_test_std = std_scale.transform(X_log_test)

On prépare le tableau final de comparaison des modèles.

In [7]:
listeModeles = []

#Modèles sans la variable ENERGYSTARScore
listeR2_SE, listeR2_log_SE = [], []
listeR2_SE_train, listeR2_log_SE_train = [], []
listeMSE_SE, listeMSE_log_SE = [], []
listeTemps_SE, listeTemps_log_SE = [], []

#Modèles avec la variable ENERGYSTARScore
listeR2, listeR2_log = [], []
listeR2_train, listeR2_log_train = [], []
listeMSE, listeMSE_log = [], []
listeTemps, listeTemps_log = [], []

# <a name="C2">Partie 2: Méthodes de Régression</a>
## <a name="C3">2.1: Régressions naïves</a> 
On va commencer par les modèles les plus simples possibles, à savoir prédire à chaque fois la valeur moyenne dans un premier temps, puis la valeur médiane dans un second temps.
### 2.1.1: Jeu sans transformation:
#### A: Sans la variable `ENERGYSTARScore`

In [8]:
listeModeles.append('Dummy_mean')
listeModeles.append('Dummy_median')

In [9]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_SE_train_std, y_SE_train)
y_SE_predict_dummy_mean = lm_dummy_mean.predict(X_SE_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(lm_dummy_mean.score(X_SE_train_std, y_SE_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_SE_train_std, y_SE_train)
y_SE_predict_dummy_median = lm_dummy_median.predict(X_SE_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(lm_dummy_median.score(X_SE_train_std, y_SE_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0295


In [10]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict_dummy_median)))
  
print("r2_score (dummy mean): {:.4f}".format(r2_score(y_SE_test, y_SE_predict_dummy_mean)))
print("r2_score (dummy median): {:.4f}".format(r2_score(y_SE_test, y_SE_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 358174.5940
Mean squared error (dummy médiane): 366688.9485
r2_score (dummy mean): -0.0008
r2_score (dummy median): -0.0246
Temps de calcul (dummy moyenne): 0.0011
Temps de calcul (dummy médiane): 0.0006


Pour les deux approches les coefficients r2 sont nuls. Ces modèles sont trop simples et ne nous donnerons pas de bons résultats de prédiction.

In [11]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict_dummy_mean),4))
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict_dummy_median),4))

listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict_dummy_mean),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict_dummy_median),4))

listeTemps_SE.append(round(time_mean,4))
listeTemps_SE.append(round(time_median,4))

#### B: Avec la variable `ENERGYSTARScore`
Regardons si l'ajout de l'Energy Star Score peut améliorer les modèles.

In [12]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train_std, y_train)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_train_std, y_train)))
listeR2_train.append(lm_dummy_mean.score(X_train_std, y_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_train_std, y_train)
y_predict_dummy_median = lm_dummy_median.predict(X_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_train_std, y_train)))
listeR2_train.append(lm_dummy_median.score(X_train_std, y_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0264


In [13]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_test, y_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_test, y_predict_dummy_median)))
  
print("r2_score (dummy mean): {:.4f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (dummy median): {:.4f}".format(r2_score(y_test, y_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 172972.6406
Mean squared error (dummy médiane): 179015.8893
r2_score (dummy mean): -0.0000
r2_score (dummy median): -0.0350
Temps de calcul (dummy moyenne): 0.0008
Temps de calcul (dummy médiane): 0.0015


Les résultats restent quasiment identiques.

In [14]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict_dummy_mean),4))
listeMSE.append(round(mean_squared_error(y_test, y_predict_dummy_median),4))

listeR2.append(round(r2_score(y_test, y_predict_dummy_mean),4))
listeR2.append(round(r2_score(y_test, y_predict_dummy_median),4))

listeTemps.append(round(time_mean,4))
listeTemps.append(round(time_median,4))

### 2.1.2: Jeu avec transformations
Regardons si les transformations logarithmiques réalisées dans la partie nettoyage peuvent améliorer les capacités de généralisation de ces premières approches naïves.
#### A: Sans la variable `ENERGYSTARScore`

In [15]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict_dummy_mean = lm_dummy_mean.predict(X_log_SE_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(lm_dummy_mean.score(X_log_SE_train_std, y_log_SE_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict_dummy_median = lm_dummy_median.predict(X_log_SE_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(lm_dummy_median.score(X_log_SE_train_std, y_log_SE_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0000


In [16]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict_dummy_median)))
  
print("r2_score (dummy mean): {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict_dummy_mean)))
print("r2_score (dummy median): {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 2.1518
Mean squared error (dummy médiane): 2.1523
r2_score (dummy mean): -0.0025
r2_score (dummy median): -0.0028
Temps de calcul (dummy moyenne): 0.0005
Temps de calcul (dummy médiane): 0.0007


In [17]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict_dummy_mean),4))
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict_dummy_median),4))

listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict_dummy_mean),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict_dummy_median),4))

listeTemps_log_SE.append(round(time_mean,4))
listeTemps_log_SE.append(round(time_median,4))

#### B: Avec la variable `ENERGYSTARScore`

In [18]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_log_train_std, y_log_train)
y_log_predict_dummy_mean = lm_dummy_mean.predict(X_log_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lm_dummy_mean.score(X_log_train_std, y_log_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_log_train_std, y_log_train)
y_log_predict_dummy_median = lm_dummy_median.predict(X_log_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lm_dummy_median.score(X_log_train_std, y_log_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0001


In [19]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_log_test, y_log_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_log_test, y_log_predict_dummy_median)))
  
print("r2_score (dummy mean): {:.4f}".format(r2_score(y_log_test, y_log_predict_dummy_mean)))
print("r2_score (dummy median): {:.4f}".format(r2_score(y_log_test, y_log_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 2.1079
Mean squared error (dummy médiane): 2.1079
r2_score (dummy mean): -0.0000
r2_score (dummy median): -0.0000
Temps de calcul (dummy moyenne): 0.0011
Temps de calcul (dummy médiane): 0.0007


In [20]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict_dummy_mean),4))
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict_dummy_median),4))

listeR2_log.append(round(r2_score(y_log_test, y_log_predict_dummy_mean),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict_dummy_median),4))

listeTemps_log.append(round(time_mean,4))
listeTemps_log.append(round(time_median,4))

Les coefficients r2 sont toujours nuls, ce qui signifie que ces modèles sont médiocres pour faire des prédictions.

## <a name="C4">Partie 2.2: Régression Linéaire</a>
Les premières approches naïves n'étant pas satisfaisantes, on va complexifier un peu le modèle.

Notre démarche, qui sera la même pour tous les prochains modèles, est la suivante:
- Trouver les meilleurs hyperparamètres (lorqu'il y en a) par recherche sur grille
- Réaliser une validation croisée sur le jeu d'entraînement pour avoir une idée des capacités de généralisation du modèle et pouvoir contrôler l'éventuel sur/sous-apprentissage
- Entraîner le modèle sur le jeu d'entraînement
- Evaluer le score du modèle sur le jeu de test
- Récupérer les différentes métriques du modèle pour pouvoir le comparer aux autres

### 2.2.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [21]:
listeModeles.append('regression_lineaire')

Pour optimiser la méthode de régression linéaire, il faut sélectionner les features qui sont corrélées avec la target mais ne sont pas corrélées entre elles. Pour cela, on peut utiliser un algorithme de sélection des x meilleures features, où x est un hyperparamètre que l'on peut trouver par une recherche sur grille.

In [22]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_SE_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_SE_train_std, y_SE_train)
    X_SE_train_fs = f_selector.transform(X_SE_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_SE_train_fs, y_SE_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.5177484851546111 est obtenu en sélectionnant les 4 meilleures features


In [23]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_SE_train_std, y_SE_train)
X_SE_train_fs = f_selector.transform(X_SE_train)
print(cross_val_score(lm, X_SE_train_fs, y_SE_train,  cv=5))

[ 0.58993871  0.74793189  0.54860045  0.1191379  -0.10010975]


In [24]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_SE_train, y_SE_train)
X_SE_train_fs = f_selector.transform(X_SE_train_std)
X_SE_test_fs = f_selector.transform(X_SE_test)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_SE_train_fs, y_SE_train)
y_SE_predict = lm.predict(X_SE_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_SE_train_fs, y_SE_train)))
listeR2_SE_train.append(lm.score(X_SE_train_fs, y_SE_train))

r2_score train: 0.7023


In [25]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))


Mean squared error: 1401482754522089.2500
r2_score: -3916104304.3819
Temps de calcul: 0.0009


In [26]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

On note de l'insatbilité lors de la validation croisée. Le r2 du test est négatif et on note un problème de sur-apprentissage lors de l'entraînement où on obtient un r2 de 0,7.

#### B: Avec la variable `ENERGYSTARScore`

In [27]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_train_std, y_train)
    X_train_fs = f_selector.transform(X_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_train_fs, y_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.7279269772180148 est obtenu en sélectionnant les 2 meilleures features


In [28]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_train_std, y_train)
X_train_fs = f_selector.transform(X_train_std)
print(cross_val_score(lm, X_train_fs, y_train,  cv=5))

[0.69574853 0.52346711 0.82733653 0.71953926 0.87354346]


In [29]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_train_std, y_train)
X_train_fs = f_selector.transform(X_train_std)
X_test_fs = f_selector.transform(X_test_std)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_train_fs, y_train)
y_predict = lm.predict(X_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_train_fs, y_train)))
listeR2_train.append(lm.score(X_train_fs, y_train))

r2_score train: 0.7868


In [30]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 21076.3836
r2_score: 0.8781
Temps de calcul: 0.0010


In [31]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Il y a encore de l'instabilité lors de la validation croisée et du sous-apprentissage avec un coefficient r2 plus grand pour le test que pour l'entraînement.

### 2.2.2: Jeu avec transformation
#### A: Sans la variable `ENERGYSTARScore`

In [32]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_log_SE_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_log_SE_train_std, y_log_SE_train)
    X_log_SE_train_fs = f_selector.transform(X_log_SE_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_log_SE_train_fs, y_log_SE_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.5441251509404189 est obtenu en sélectionnant les 56 meilleures features


In [33]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_SE_train_std, y_log_SE_train)
X_log_SE_train_fs = f_selector.transform(X_log_SE_train_std)
print(cross_val_score(lm, X_log_SE_train_fs, y_log_SE_train,  cv=5))

[0.58019477 0.53462208 0.51665598 0.49856654 0.59058638]


In [34]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_SE_train_std, y_log_SE_train)
X_log_SE_train_fs = f_selector.transform(X_log_SE_train_std)
X_log_SE_test_fs = f_selector.transform(X_log_SE_test_std)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_log_SE_train_fs, y_log_SE_train)
y_log_SE_predict = lm.predict(X_log_SE_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_log_SE_train_fs, y_log_SE_train)))
listeR2_log_SE_train.append(lm.score(X_log_SE_train_fs, y_log_SE_train))

r2_score train: 0.5602


In [35]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.9376
r2_score: 0.5631
Temps de calcul: 0.0077


In [36]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Les transformations logarithmiques ont stabilisé le modèle et amoindri les effets de sur/sous-apprentissage. Le coefficient n'est toutefois pas très élevé.

#### B: Avec la variable `ENERGYSTARScore`

In [37]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_log_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_log_train_std, y_log_train)
    X_log_train_fs = f_selector.transform(X_log_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_log_train_fs, y_log_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.6847700531365462 est obtenu en sélectionnant les 59 meilleures features


In [38]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_train_std, y_log_train)
X_log_train_fs = f_selector.transform(X_log_train_std)
print(cross_val_score(lm, X_log_train_fs, y_log_train,  cv=5))

[0.70362102 0.72509661 0.73182637 0.65895942 0.60434685]


In [39]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_train_std, y_log_train)
X_log_train_fs = f_selector.transform(X_log_train_std)
X_log_test_fs = f_selector.transform(X_log_test_std)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_log_train_fs, y_log_train)
y_log_predict = lm.predict(X_log_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_log_train_fs, y_log_train)))
listeR2_log_train.append(lm.score(X_log_train_fs, y_log_train))

r2_score train: 0.6995


In [40]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6198
r2_score: 0.7060
Temps de calcul: 0.0096


In [41]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

L'ajout de la varibale `ENERGYSTARScore` produit un gain d'environ 0,1 sur le coefficient r2.

## <a name="C5">2.3: Régression Ridge</a>
Pour éviter l'éventuel sur-appretissage du modèle de régression linéaire, on va utiliser une technique de régularisation afin de pouvoir contrôler en plus de l'erreur du modèle, sa complexité.
### 2.3.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [42]:
listeModeles.append("regression_ridge")

In [43]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_SE_train_std, y_SE_train)
alpha = reg.alpha_

In [44]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_SE_train_std, y_SE_train,  cv=5))

[0.5312539  0.79477082 0.63698635 0.17072091 0.12360071]


In [45]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_SE_train_std, y_SE_train)
y_SE_predict = lmr.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(lmr.score(X_SE_train_std, y_SE_train))

r2_score train: 0.7723


In [46]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 137001.5091
r2_score: 0.6172
Temps de calcul: 0.0460


In [47]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instabilité et sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [48]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_train_std, y_train)
alpha = reg.alpha_

In [49]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_train_std, y_train,  cv=5))

[0.81551151 0.64312435 0.52246476 0.78507081 0.73293539]


In [50]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_train_std, y_train)
y_predict = lmr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_train_std, y_train)))
listeR2_train.append(lmr.score(X_train_std, y_train))

r2_score train: 0.8696


In [51]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 18933.6237
r2_score: 0.8905
Temps de calcul: 0.0441


In [52]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Encore de l'instabilité.

### 2.3.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [53]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_log_SE_train_std, y_log_SE_train)
alpha = reg.alpha_

In [54]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.59626895 0.55566835 0.55257966 0.49333762 0.62017857]


In [55]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = lmr.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(lmr.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.6189


In [56]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.9002
r2_score: 0.5806
Temps de calcul: 0.0560


In [57]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Moins d'instabilité mais un peu de sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [58]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_

In [59]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_log_train_std, y_log_train,  cv=5))

[0.71635446 0.73497184 0.75118349 0.68745106 0.631709  ]


In [60]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_log_train_std, y_log_train)
y_log_predict = lmr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lmr.score(X_log_train_std, y_log_train))

r2_score train: 0.7353


In [61]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.5777
r2_score: 0.7259
Temps de calcul: 0.0442


In [62]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

La variable `ENERGYSTARScore` et les transformations logarithmiques apportent un gain intéressant.

## <a name="C6">2.4: Régression Lasso</a>
Pour tenter d'améliorer la capacité de généralisation de la régression ridge on peut ajouter en plus une réduction dimensionnelle, notamment pour supprimer les éventuelles variables corrélées.
### 2.4.1: Jeu sans transformation:
#### A: Sans la variable `ENERGYSTARScore`

In [63]:
listeModeles.append('regression_lasso')

In [64]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_SE_train_std, y_SE_train)
alpha = reg.alpha_

In [65]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_SE_train_std, y_SE_train,  cv=5))

[0.55993696 0.79348154 0.66197107 0.16194907 0.16626856]


In [66]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_SE_train_std, y_SE_train)
y_SE_predict = lml.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(lml.score(X_SE_train_std, y_SE_train))

r2_score train: 0.7707


In [67]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 136421.7538
r2_score: 0.6188
Temps de calcul: 0.6443


In [68]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instabilité et sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [69]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_train_std, y_train)
alpha = reg.alpha_

In [70]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_train_std, y_train,  cv=5))

[0.81622473 0.63815021 0.52475489 0.78516628 0.72836054]


In [71]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_train_std, y_train)
y_predict = lml.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_train_std, y_train)))
listeR2_train.append(lml.score(X_train_std, y_train))

r2_score train: 0.8691


In [72]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 19011.7019
r2_score: 0.8901
Temps de calcul: 0.2231


In [73]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

### 2.4.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [74]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_log_SE_train_std, y_log_SE_train)
alpha = reg.alpha_

In [75]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.59762522 0.56724607 0.56389735 0.51362988 0.62662846]


In [76]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = lml.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(lml.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.6112


In [77]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.8892
r2_score: 0.5857
Temps de calcul: 0.1634


In [78]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Plus stable mais sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [79]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_

In [80]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_log_train_std, y_log_train,  cv=5))

[0.72501134 0.73874435 0.75467083 0.68911725 0.63029026]


In [81]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_log_train_std, y_log_train)
y_log_predict = lml.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lml.score(X_log_train_std, y_log_train))

r2_score train: 0.7322


In [82]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.5743
r2_score: 0.7275
Temps de calcul: 0.1625


In [83]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Stable et avec un coefficient r2 qui commence à devenir intéressant. Continuons à augmenter la complexité des modèles testés.

## <a name="C7">2.5: Régression ElasticNet</a>
La régression ridge permet d'éviter le sur-appretissage avec une solution unique lorsque la régression lasso permet d'avoir un modèle plus parcimonieux mais instable. Il exite un modèle de prédiction linéaire intermédiaire qui combine les deux approches: la régression ElasticNet.
### 2.5.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [84]:
listeModeles.append('regression_elasticNet')

In [85]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_SE_train_std, y_SE_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [86]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha, l1_ratio)
print(cross_val_score(lml, X_SE_train_std, y_SE_train,  cv=5))

C:\Users\vbour\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass l1_ratio=0.9 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 73701531.41355705, tolerance: 87370.6021122844
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 68436105.52169967, tolerance: 71431.61208464052
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Object

[0.52988576 0.7947272  0.6625462  0.1717644  0.12000768]


C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 85470793.02432205, tolerance: 98906.00717406295
  model = cd_fast.enet_coordinate_descent(


In [87]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha, l1_ratio).fit(X_SE_train_std, y_SE_train)
y_SE_predict = lmE.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(lmE.score(X_SE_train_std, y_SE_train))

C:\Users\vbour\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass l1_ratio=0.9 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


r2_score train: 0.7666


In [88]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 137229.8957
r2_score: 0.6165
Temps de calcul: 0.2348


In [89]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instable et sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [90]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_train, y_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [91]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha, l1_ratio)
print(cross_val_score(lml, X_train, y_train,  cv=5))

C:\Users\vbour\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass l1_ratio=0.1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 64738855.86143594, tolerance: 105851.56566027328
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 55363525.14301769, tolerance: 96312.87273523679
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Obje

[0.81333482 0.63622472 0.52475404 0.7810649  0.73791428]


C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74321498.95116487, tolerance: 95634.16481359571
  model = cd_fast.enet_coordinate_descent(


In [92]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha, l1_ratio).fit(X_train, y_train)
y_predict = lmE.predict(X_test)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_train_std, y_train)))
listeR2_train.append(lmE.score(X_train_std, y_train))

r2_score train: -0.0390


C:\Users\vbour\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass l1_ratio=0.1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [93]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 38125.0236
r2_score: 0.7796
Temps de calcul: 0.0390


In [94]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Instable.

### 2.5.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [95]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_log_SE_train_std, y_log_SE_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [96]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
print(cross_val_score(lmE, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.59758607 0.56720346 0.56385993 0.51368142 0.62657506]


In [97]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = lmE.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(lmE.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.6111


In [98]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.8893
r2_score: 0.5857
Temps de calcul: 0.1922


In [99]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Plus stable mais avec un léger sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [100]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [101]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
print(cross_val_score(lmE, X_log_train_std, y_log_train,  cv=5))

[0.7228949  0.73638761 0.75258376 0.688962   0.64277054]


In [102]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio).fit(X_log_train_std, y_log_train)
y_log_predict = lmE.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lmE.score(X_log_train_std, y_log_train))

r2_score train: 0.7311


In [103]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.5779
r2_score: 0.7258
Temps de calcul: 0.1275


In [104]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

De meilleurs résultats avec le jeu transformé et la variable `ENERGYSTARScore`

# <a name="C8">Partie 3: Méthodes ensemblistes</a>
Le principe des méthodes ensemblistes est de combiner un certain nombre de modèles avec des performances faibles pour obtenir un modèle prédictif plus efficace en combinant les résultats via la moyenne par exemple pour les problèmes de régression.
## <a name="C9">3.1: Bagging</a>
Le bagging fonctionne en 3 étapes:
- On génère N nouveaux jeux de données en tirant aléatoirement (avec remise) chaque observation dans le jeu de données initial
- On entraîne un apprenant faible sur chaque jeu, ici un arbre de décision
- On effectue une prédiction en faisant la moyenne des prédictions des apprenants faibles
### 3.1.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [105]:
listeModeles.append('Bagging')

In [106]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_SE_train_std, y_SE_train)
n_estimators = reg.best_params_.get('n_estimators')

In [107]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_SE_train_std, y_SE_train,  cv=5))

[0.62310101 0.64545677 0.8437748  0.18567495 0.66161423]


In [108]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_SE_train_std, y_SE_train)
y_SE_predict = bagging.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(bagging.score(X_SE_train_std, y_SE_train))

r2_score train: 0.9372


In [109]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 37705.7766
r2_score: 0.8946
Temps de calcul: 11.8310


In [110]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instable et sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [111]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')

In [112]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_train_std, y_train,  cv=5))

[0.52441095 0.89749702 0.87620746 0.92242821 0.83382224]


In [113]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_train_std, y_train)
y_predict = bagging.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_train_std, y_train)))
listeR2_train.append(bagging.score(X_train_std, y_train))

r2_score train: 0.9773


In [114]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 24525.4067
r2_score: 0.8582
Temps de calcul: 10.6939


In [115]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Encore un peu d'instabilité et de sur-apprentissage mais un coefficient r2 qui devient de plus en plus intéressant.

### 3.1.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [116]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_SE_train_std, y_log_SE_train)
n_estimators = reg.best_params_.get('n_estimators')

In [117]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.74796341 0.70884586 0.71351567 0.68562176 0.75456993]


In [118]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = bagging.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(bagging.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.9674


In [119]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.4529
r2_score: 0.7890
Temps de calcul: 10.6800


In [120]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Instable et sur-apprentissage.
#### B: Avec la variable `ENERGYSTARScore`

In [121]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')

In [122]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_log_train_std, y_log_train,  cv=5))

[0.90693107 0.91527297 0.90657714 0.90341107 0.90876143]


In [123]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_log_train_std, y_log_train)
y_log_predict = bagging.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(bagging.score(X_log_train_std, y_log_train))

r2_score train: 0.9884


In [124]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.2039
r2_score: 0.9033
Temps de calcul: 11.5592


In [125]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Encore du sur-apprentissage mais un coefficient de test r2 très bon.

## <a name="C10">3.2: Random Forest</a>
Les forêts aléatoires reposent sur le même principe que le bagging mais en sélectionnant un nombre plus restreint de features afin de créer des arbres les plus différents possible les uns des autres et limiter ainsi le sur-apprentissage.
### 3.2.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [126]:
listeModeles.append('RandomForest')

In [127]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_SE_train_std, y_SE_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [128]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_SE_train_std, y_SE_train,  cv=5))

[0.59906129 0.65960767 0.85155887 0.19527017 0.61682125]


In [129]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_SE_train_std, y_SE_train)
y_SE_predict = forest.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(forest.score(X_SE_train_std, y_SE_train))

r2_score train: 0.9170


In [130]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 41945.5092
r2_score: 0.8828
Temps de calcul: 16.3511


In [131]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instable.

#### B: Avec la variable `ENERGYSTARScore`

In [132]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [133]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_train_std, y_train,  cv=5))

[0.35789499 0.91190238 0.83741522 0.93509273 0.82453455]


In [134]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_train_std, y_train)
y_predict = forest.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_train_std, y_train)))
listeR2_train.append(forest.score(X_train_std, y_train))

r2_score train: 0.9721


In [135]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 18078.9690
r2_score: 0.8955
Temps de calcul: 0.7909


In [136]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Encore un petit peu d'instabilité.

### 3.2.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [137]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_SE_train_std, y_log_SE_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [138]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.74675524 0.70850505 0.70748871 0.68925363 0.74999905]


In [139]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_log_SE_train_std, 
                                                                                                   y_log_SE_train)
y_log_SE_predict = forest.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(forest.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.9676


In [140]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.4526
r2_score: 0.7892
Temps de calcul: 8.2552


In [141]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Sur-apprentissage marqué sur le jeu d'entraînement.

#### B: Avec la variable `ENERGYSTARScore`

In [142]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [143]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_log_train_std, y_log_train,  cv=5))

[0.90695144 0.91478722 0.90790295 0.90376693 0.91053677]


In [144]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_log_train_std, y_log_train)
y_log_predict = forest.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(forest.score(X_log_train_std, y_log_train))

r2_score train: 0.9883


In [145]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.2007
r2_score: 0.9048
Temps de calcul: 6.9382


In [146]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Du sur-apprentissage mais un très bon coefficient de test r2.

## <a name="C11">3.3: Gradient Boosting</a>
Les méthodes ensemblistes utilisées plus haut sont dites parallèles, c'est-à-dire que les apprenants faibles sont entraînés indépendamment les uns des autres. Le Gradient Boosting au contraire est une méthode ensembliste dite séquentielle. Les apprenants faibles sont entraînés les uns à la suite des autres, notamment en mettant un peu plus l'accent sur les observations qui ont provoqué des erreurs sur l'apprenant précédent.
### 3.3.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [147]:
listeModeles.append('GradientBoosting')

In [148]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_SE_train_std, y_SE_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [149]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_SE_train_std, y_SE_train,  cv=5))

[0.59403661 0.61078895 0.87304181 0.16245609 0.69656671]


In [150]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_SE_train_std, 
                                                                                                               y_SE_train)
y_SE_predict = boosting.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(boosting.score(X_SE_train_std, y_SE_train))

r2_score train: 0.9699


In [151]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 29667.2679
r2_score: 0.9171
Temps de calcul: 2.2072


In [152]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Instable.

#### B: Avec la variable `ENERGYSTARScore`

In [153]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [154]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_train_std, y_train,  cv=5))

[0.69856233 0.72316543 0.90278415 0.9770997  0.86020388]


In [155]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_train_std, 
                                                                                                               y_train)
y_predict = boosting.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_train_std, y_train)))
listeR2_train.append(boosting.score(X_train_std, y_train))

r2_score train: 0.9944


In [156]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 20528.8318
r2_score: 0.8813
Temps de calcul: 2.4952


In [157]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Encore un peu d'instabilité et de sur-apprentissage.
### 3.3.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [158]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_SE_train_std, y_log_SE_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [159]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.66097648 0.61908549 0.6055046  0.59851342 0.67647783]


In [160]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_log_SE_train_std, 
                                                                                                               y_log_SE_train)
y_log_SE_predict = boosting.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(boosting.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.7172


In [161]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.7237
r2_score: 0.6628
Temps de calcul: 2.2399


In [162]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Stable, un peu de sur-apprentissage mais score moins bon que ceux rencontrés un peu plus haut.
#### B: Avec la variable `ENERGYSTARScore`

In [163]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [164]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_log_train_std, y_log_train,  cv=5))

[0.89148236 0.89608588 0.89615496 0.88532838 0.89142975]


In [165]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_log_train_std, 
                                                                                                               y_log_train)
y_log_predict = boosting.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(boosting.score(X_log_train_std, y_log_train))

r2_score train: 0.9229


In [166]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.2514
r2_score: 0.8807
Temps de calcul: 2.6471


In [167]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Un peu de sur-apprentissage mais un bon coefficient de test r2.

# <a name="C12">Partie 4: Méthodes non linéaires</a>
Beaucoup de phénomènes n'étant pas linéaires, on peut se retrouver dans une impasse si l'on cherche à effectuer des prédictions à l'aide de méthodes linéaires comme celles utilisées jusqu'à maintenant. On va ainsi tester des méthodes non linéaires pour voir si l'on peut obtenir de meilleures performances de généralisation.
## <a name="C13">4.1: SVM à noyau</a>
### 4.1.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [168]:
listeModeles.append('SVM_noyau')

In [169]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_SE_train_std, y_SE_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [170]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_SE_train_std, y_SE_train,  cv=5))

[0.35406637 0.29045803 0.48996457 0.09951381 0.40991838]


In [171]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_SE_train_std, y_SE_train)
y_SE_predict = svr.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(svr.score(X_SE_train_std, y_SE_train))

r2_score train: 0.4255


In [172]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 202855.0794
r2_score: 0.4332
Temps de calcul: 11.2614


In [173]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Stable mais score faible.

#### B: Avec la variable `ENERGYSTARScore`

In [174]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [175]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_train_std, y_train,  cv=5))

[0.6068019  0.36937648 0.57090053 0.4623726  0.09475441]


In [176]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_train_std, y_train)
y_predict = svr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_train_std, y_train)))
listeR2_train.append(svr.score(X_train_std, y_train))

r2_score train: 0.5585


In [177]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 58454.3501
r2_score: 0.6620
Temps de calcul: 12.5920


In [178]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Instable.

### 4.1.2: Jeu avec Transformations
#### A: Sans la variable `ENERGYSTARScore`

In [179]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_SE_train_std, y_log_SE_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [180]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.6532418  0.56274424 0.5725725  0.57006831 0.63639176]


In [181]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = svr.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(svr.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.7553


In [182]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.8076
r2_score: 0.6238
Temps de calcul: 5.2455


In [183]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Plus stable mais avec du sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [184]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [185]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_log_train_std, y_log_train,  cv=5))

[0.79836458 0.8155788  0.79694013 0.78028944 0.8037407 ]


In [186]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_log_train_std, y_log_train)
y_log_predict = svr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(svr.score(X_log_train_std, y_log_train))

r2_score train: 0.9171


In [187]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.4020
r2_score: 0.8093
Temps de calcul: 5.3700


In [188]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Stable mais avec un important problème de sur-apprentissage.

## <a name="C14">4.2: Régression ridge à noyau</a>
Le principe de fonctionnement est le même que pour la SVM à noyau, seule la fonction de perte utilisée diffère.
### 4.2.1: Jeu sans transformation
#### A: Sans la variable `ENERGYSTARScore`

In [189]:
listeModeles.append('Ridge_noyau')

In [190]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_SE_train_std, y_SE_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [191]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_SE_train_std, y_SE_train,  cv=5))

[0.72203671 0.52175012 0.8089789  0.13752831 0.517304  ]


In [192]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_SE_train_std, y_SE_train)
y_SE_predict = krr.predict(X_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_SE_train_std, y_SE_train)))
listeR2_SE_train.append(krr.score(X_SE_train_std, y_SE_train))

r2_score train: 0.9862


In [193]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_SE_test, y_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_SE_test, y_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 38505.0971
r2_score: 0.8924
Temps de calcul: 4.3520


In [194]:
#Enregistrement des métriques
listeMSE_SE.append(round(mean_squared_error(y_SE_test, y_SE_predict),4))
listeR2_SE.append(round(r2_score(y_SE_test, y_SE_predict),4))
listeTemps_SE.append(round(time,4))

Modèle instable avec soucis de sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [195]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [196]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_train_std, y_train,  cv=5))

[0.91764413 0.5827606  0.62343971 0.90180449 0.06762842]


In [197]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_train_std, y_train)
y_predict = krr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_train_std, y_train)))
listeR2_train.append(krr.score(X_train_std, y_train))

r2_score train: 0.9974


In [198]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 23103.8239
r2_score: 0.8664
Temps de calcul: 4.1490


In [199]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Résultats semblables avec ou sans la variable `ENERGYSTARScore`

### 4.2.2: Jeu avec transformations
#### A: Sans la variable `ENERGYSTARScore`

In [200]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_SE_train_std, y_log_SE_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [201]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_log_SE_train_std, y_log_SE_train,  cv=5))

[0.40507398 0.2977881  0.25495386 0.38579244 0.38276704]


In [202]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_log_SE_train_std, y_log_SE_train)
y_log_SE_predict = krr.predict(X_log_SE_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_log_SE_train_std, y_log_SE_train)))
listeR2_log_SE_train.append(krr.score(X_log_SE_train_std, y_log_SE_train))

r2_score train: 0.8328


In [203]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_SE_test, y_log_SE_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_SE_test, y_log_SE_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 1.1702
r2_score: 0.4548
Temps de calcul: 4.1807


In [204]:
#Enregistrement des métriques
listeMSE_log_SE.append(round(mean_squared_error(y_log_SE_test, y_log_SE_predict),4))
listeR2_log_SE.append(round(r2_score(y_log_SE_test, y_log_SE_predict),4))
listeTemps_log_SE.append(round(time,4))

Trop de sur-apprentissage.

#### B: Avec la variable `ENERGYSTARScore`

In [205]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [206]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_log_train_std, y_log_train,  cv=5))

[0.53616015 0.59709723 0.44331084 0.63957292 0.63648376]


In [207]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_log_train_std, y_log_train)
y_log_predict = krr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(krr.score(X_log_train_std, y_log_train))

r2_score train: 0.9590


In [208]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6256
r2_score: 0.7032
Temps de calcul: 4.5472


In [209]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Résultats semblables avec ou sans la variable `ENERGYSTARScore`

# <a name="C15">Partie 5: Sélection du modèle de prédiction</a>
Il est maintenant temps de choisir parmis les modèles testés, lequel semble le plus pertinent et adapté pour prédire les émissions de CO2 des bâtiments de la ville de Seattle. Notre choix s'appuiera sur la capacité de généralisation du modèle, son temps de calcul et les contraintes liées à la sélection des hyperparamètres.
## 5.1: Jeu non transformé
Commençons par regrouper les scores des modèles dans des tableaux pour faciliter les comparaisons.

In [214]:
#Jeu non transformé sans la variable ENERGYSTARScore
tab = np.array([listeMSE_SE, listeR2_SE_train, listeR2_SE, listeTemps_SE])
df_SE = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test',  'Temps_Calcul'], columns=listeModeles)
df_SE

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,358174.5940,366688.94850,1.401483e+15,137001.509100,136421.753800,137229.895700,37705.776600,41945.509200,29667.267900,202855.079400,38505.09710
r2 train,0.0000,-0.02948,7.022766e-01,0.772278,0.770675,0.766567,0.937171,0.917034,0.969869,0.425463,0.98619
r2 test,-0.0008,-0.02460,-3.916104e+09,0.617200,0.618800,0.616500,0.894600,0.882800,0.917100,0.433200,0.89240
Temps_Calcul,0.0011,0.00060,9.000000e-04,0.046000,0.644300,0.234800,11.831000,16.351100,2.207200,11.261400,4.35200


In [215]:
#Jeu non transformé avec la variable ENERGYSTARScore
tab = np.array([listeMSE, listeR2_train, listeR2, listeTemps])
df = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test', 'Temps_Calcul'], columns=listeModeles)
df

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,172972.6406,179015.88930,21076.383600,18933.623700,19011.701900,38125.023600,24525.406700,18078.969000,20528.831800,58454.350100,23103.823900
r2 train,0.0000,-0.02643,0.786826,0.869571,0.869061,-0.039018,0.977251,0.972063,0.994421,0.558499,0.997421
r2 test,-0.0000,-0.03500,0.878100,0.890500,0.890100,0.779600,0.858200,0.895500,0.881300,0.662000,0.866400
Temps_Calcul,0.0008,0.00150,0.001000,0.044100,0.223100,0.039000,10.693900,0.790900,2.495200,12.592000,4.149000


L'ajout de la variable `ENERGYSTARscore` apporte un gain notable pour les méthodes de régression. </br>
Les méthodes qui semblent à première vue intéressantes sont les méthodes ensemblistes et la régression ridge à noyau. Voyons si on peut obtenir mieux avec ke jeu de données transformé.

## 5.2: Jeu transformé
Commençons par regrouper les scores des modèles dans des tableaux pour faciliter les comparaisons.

In [216]:
#Jeu transformé sans la variable ENERGYSTARScore
tab = np.array([listeMSE_log_SE, listeR2_log_SE_train, listeR2_log_SE, listeTemps_log_SE])
df_log_SE = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test', 'Temps_Calcul'], columns=listeModeles)
df_log_SE

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,2.1518,2.152300,0.937600,0.900200,0.889200,0.889300,0.452900,0.452600,0.72370,0.807600,1.170200
r2 train,0.0000,-0.000006,0.560183,0.618862,0.611196,0.611147,0.967403,0.967597,0.71718,0.755331,0.832794
r2 test,-0.0025,-0.002800,0.563100,0.580600,0.585700,0.585700,0.789000,0.789200,0.66280,0.623800,0.454800
Temps_Calcul,0.0005,0.000700,0.007700,0.056000,0.163400,0.192200,10.680000,8.255200,2.23990,5.245500,4.180700


In [217]:
#Jeu transformé avec la variable ENERGYSTARScore
tab = np.array([listeMSE_log, listeR2_log_train, listeR2_log, listeTemps_log])
df_log = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test', 'Temps_Calcul'], columns=listeModeles)
df_log

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,2.1079,2.107900,0.61980,0.577700,0.574300,0.577900,0.203900,0.200700,0.251400,0.402000,0.625600
r2 train,0.0000,-0.000075,0.69949,0.735262,0.732216,0.731081,0.988381,0.988336,0.922928,0.917093,0.958989
r2 test,-0.0000,-0.000000,0.70600,0.725900,0.727500,0.725800,0.903300,0.904800,0.880700,0.809300,0.703200
Temps_Calcul,0.0011,0.000700,0.00960,0.044200,0.162500,0.127500,11.559200,6.938200,2.647100,5.370000,4.547200


Globalement, les transformations logarithmiques ne conduisent pas à un gain significatif sur le score des modèles.

## 5.3: Conclusion

Tout d'abord, les transformations logarithmiques donnent de bons résultats mais le jeu non transformé également. Toutefois, on a souvent obtenu des modèles plus stables avec le jeu de données transformé. Il semble donc plus judicieux de conserver le jeu de données transformé.

Dans un second temps, on a pu remarquer que l'ajout de la variable `ENERGYSTARScore` améliorait les scores de généralisation des méthodes régressives, mais qu'on pouvait également avoir de très bons modèles de prédiction sans son utilisation. Cependant, l'ajout de cette variable a souvent permis de stabiliser les modèles. Notre conseil pour la ville de Seattle est donc de continuer le calcul de cette donnée car elle permet d'obtenir par la suite des modèles de prédiction plus robustes.

Il ne nous reste donc plus qu'à faire un choix entre les trois méthodes ensemblistes et la régression ridge à noyau </br>
La régression ridge à un score r2 nettement inférieur aux méthodes ensemblistes, on va donc l'écarter. </br>
Le bagging et les forêts aléatoires ont de meilleurs score r2 que le gradient boosting mais avec des temps de calcul plus long et un sur-apprentissage plus important, on va également les écartet. 


Finalement, on fait le choix du **Gradient Boosting** sur le jeu transformé et avec l'utilisation de la variable `ENERGYSTARScore` qui donne de très bons résultats avec un temps d'exécution court. </br>
Pour tenter d'améliorer encore plus ce modèle, on peut essayer d'optimiser les deux hyperparamètres **n_estimators** et **learning_rate** sur des plages de valeurs plus larges, notamment pour le nombre d'apprenants faibles **n_estimators**.